In [1]:
import pandas as pd
import numpy as np
from breeze_connect import BreezeConnect
import datetime as dt
import base64 
import socketio



In [2]:
import logging

# Create a logger object
log = logging.getLogger()
log.setLevel(logging.DEBUG)  # Set the root logger level to DEBUG

# Create a file handler for saving all logs
file_handler = logging.FileHandler('log_file.txt')
file_handler.setLevel(logging.DEBUG)  # Save all levels to file

# Create a console handler for printing logs to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set console handler level to INFO

# Define format for both handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add the handlers to the logger
log.addHandler(file_handler)
log.addHandler(console_handler)




In [3]:
with open('ticks.txt', 'a+') as file:
        # Perform a for loop
        file.write(f"open,high,low,close")

def records_ticks(datetime,open,high,low,close):
    # Open the file in append mode (create if it doesn't exist)
    with open('ticks.txt', 'a+') as file:
        # Perform a for loop
        file.write(f"{open},{high},{low},{close}")

In [4]:
Key = '73=63fT44781tY4C42L0i3491201v23A'
Secret = '11331XF5fm6c5&Y3K2Q99lp28347480+'
api_session = '37507590'

In [5]:

# Initialize SDK

breeze = BreezeConnect(api_key=Key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

#print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret=Secret,
                        session_token=api_session)


# Generate ISO8601 Date/DateTime String
import datetime
iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'

In [6]:
stock = 'CNXBAN'
s_type = 'futures'
exp = '27-Mar-2024'

In [ ]:
breeze.ws_connect()

def on_ticks(ticks):
    print("Ticks: {}".format(ticks))

breeze.on_ticks = on_ticks

breeze.subscribe_feeds(exchange_code="NFO", stock_code=stock, product_type=s_type, expiry_date=exp)


In [7]:
old_price_long = 0
old_price_short = 0
target_prencentage = 0.1
stop_loss_precentage = 0.2 
stop_loss_long = False
stop_loss_short = False
long_hold = False
short_hold = False
trade_book = []
qty = 30
ticks = []





log.debug(f'''Initiating All Variables Initial Values are :-
            old_price_long = {old_price_long}
            old_price_short = {old_price_short}
            target_prencantage = {target_prencentage}
            stop_loss_precentage = {stop_loss_precentage}
            stop_loss_long = {stop_loss_long}
            stop_loss_short = {stop_loss_short}
            long_hold = {long_hold}
            short_hold = {short_hold}
            trade_book = {trade_book}
            qty = {qty}''')


In [ ]:
breeze.ws_connect()

def on_ticks(ticks):
    print(type(ticks),'\n')
    
    
    global old_price_long,old_price_short,target_prencentage,stop_loss_precentage,long_hold,short_hold,trade_book,qty,stop_loss_long,stop_loss_short

    log.debug(f"variables values when running on_ticks getting ticks :- old_price_long : {old_price_long},old_price_short : {old_price_short},target_prencantage : {target_prencentage},stop_loss_precentage : {stop_loss_precentage},long_hold : {long_hold},short_hold : {short_hold},trade_book : {trade_book},qty : {qty}")
    #records_ticks(ticks['datetime'],ticks['open'],ticks['high'],ticks['low'],ticks['close'])

    log.info(f"'*******,datetime : {ticks['datetime']}, open : {ticks['open']}, high : {ticks['high']}, low : {ticks['low']}, close : {ticks['close']},'*******\n")
    
    current_price = float(ticks['close'])
    date = ticks['datetime']
    
    per_change_long = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    per_change_short = ((current_price-old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    price_change_long = current_price-old_price_long
    price_change_short = current_price-old_price_short
    log.info(f"% Change in Price for Long position = {per_change_long}")
    log.info(f"% Change in Price for Short position = {per_change_short} NOTE :: Negetive means profit in short trade same formula has been used to both long and short") 
    
    if (per_change_long >= target_prencentage) and long_hold :

        log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
        long_hold = False
        log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
        profit_long = price_change_long
        log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")

    if per_change_long <= -stop_loss_precentage and long_hold:

        log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
        long_hold = False
        log.debug(f"Assigned False value to long_hold : {long_hold}")
        stop_loss_long = True
        log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
        loss = current_price-old_price_long
        log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    if (not long_hold) and (not stop_loss_long):

        log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
        long_hold = True

        log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0})


    
    if (per_change_short <= -target_prencentage) and short_hold:
        log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
        short_hold = False
        log.debug(f'Set short_hold to False new Value = {short_hold}')
        profit_short = old_price_short-current_price 
        log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
        log.info(f"Booked a profit of {profit_long} * {qty} = {profit_long*qty}")
        order_value = qty*current_price
        
        trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty})



    if per_change_short >= stop_loss_precentage and short_hold:
        log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
        short_hold = False
        log.debug('Assigned new value to short_hold False : {False}')
        stop_loss_short = True
        log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
        loss_on_short = old_price_short-current_price
        log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
        
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss*qty})

    
    if (not short_hold) and (not stop_loss_short):
        
        log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
        short_hold = True
        log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0})
    
    

    

    '''if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            print(f'Ending Trade for day {current_price}. profit = {(current_price-old_price_long)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stt_rs = order_value*stt
            charges = brokerage+stt_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})


        if short_hold:
            short_hold = False
            profit_short = -current_price+old_price_short
            print(f'Ending Short at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stamp_rs = order_value*stamp
            charges = brokerage+stamp_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
            
    
        break'''




    
    


breeze.on_ticks = on_ticks
breeze.subscribe_feeds(exchange_code="NFO", stock_code=stock, product_type=s_type, expiry_date=exp,interval='1minute')


{'message': 'Stock CNXBAN subscribed successfully'}

[2024-03-22 12:53:59,474][INFO] - '*******,datetime : 2024-03-22 12:53:00, open : 46906.8, high : 46918.45, low : 46903.4, close : 46916.0,'*******

[2024-03-22 12:53:59,476][INFO] - % Change in Price for Long position = 0
[2024-03-22 12:53:59,477][INFO] - % Change in Price for Short position = 0 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-22 12:53:59,480][INFO] - Going Long Since not Holding and neither stoploss is triggered values :: long_hold : False, stop_loss_long : False
[2024-03-22 12:53:59,481][INFO] - Buying at a price 46916.0 * 30. total buy price 1407480.0 
 
[2024-03-22 12:53:59,483][INFO] - Going Short since Both short_hold : False and stop_loss_short : False are False. 
[2024-03-22 12:53:59,485][INFO] - Shorting at a price 46916.0. total buy price 1407480.0 
 


<class 'dict'> 



[2024-03-22 12:55:03,322][INFO] - '*******,datetime : 2024-03-22 12:54:00, open : 46916.0, high : 46944.95, low : 46915.0, close : 46936.95,'*******

[2024-03-22 12:55:03,325][INFO] - % Change in Price for Long position = 0.044654275726824726
[2024-03-22 12:55:03,327][INFO] - % Change in Price for Short position = 0.044654275726824726 NOTE :: Negetive means profit in short trade same formula has been used to both long and short


<class 'dict'> 



...

<class 'dict'> 



[2024-03-22 13:10:00,127][INFO] - '*******,datetime : 2024-03-22 13:09:00, open : 46932.45, high : 46950.0, low : 46926.45, close : 46949.0,'*******

[2024-03-22 13:10:00,130][INFO] - % Change in Price for Long position = -0.03406792292132439
[2024-03-22 13:10:00,133][INFO] - % Change in Price for Short position = 0.07033847727854037 NOTE :: Negetive means profit in short trade same formula has been used to both long and short


<class 'dict'> 



[2024-03-22 13:11:01,252][INFO] - '*******,datetime : 2024-03-22 13:10:00, open : 46949.0, high : 46955.0, low : 46945.0, close : 46951.3,'*******

[2024-03-22 13:11:01,255][INFO] - % Change in Price for Long position = -0.02917065900137781
[2024-03-22 13:11:01,257][INFO] - % Change in Price for Short position = 0.07524085599795999 NOTE :: Negetive means profit in short trade same formula has been used to both long and short


<class 'dict'> 



In [25]:
breeze.ws_disconnect()

[{'message': 'socket server for rate refresh  has been disconnected.'},
 {'message': 'socket server for OHLCV Streaming has been disconnected.'},
 {'message': 'socket server is not connected to order refresh.'}]